"Descent into Collective Entanglement"

This thesis work is to explore the possibility of faster calculations for calculating "Collectibility" for a given density matrix for a given multipartite system. In this work, we mainly focus on the 3*3 system.

We we wilm be trying out gradient descent algorithms for the purpose of finding the optimum "separable state set" to get a minimum value of collectibility. This minima, in return, will indicate if the system is entangled.

We will start with testing on known classes of density matrices (eg, werner states) which have known bounds as a sanity check of the approach.

Steps - 
1. Function working with unitary matrix. It will have a function to calculate random unitary matrix using Zyczkowski's method. 
2. Moreover, it needs to calculate the partial differential of U wrt an individual parameter. And then return the gradient for the whole parameters vector. 
3. Decide on the step size.
4. Work on generating density matrices - Random Density matrices and werner states.
5. Work out the collectibility function for given parameters.
6. After calling the gradient calculating function and deciding on the step size, we calculate the value of Collectibility for the next step.
7. We find the local minima, and do this procedure more times, to search fort he global minima.
8. Visualize the results for plotting graphs and getting a visual sense of the work.
9. Compare with brute force methods by searching for the optimum solution (minima) by random search.

In [68]:
# Library imports

import random
import cmath,math
from pprint import pprint
import copy
import time
import itertools

#numpy and matplot
import matplotlib.pyplot as plt
import numpy as np

#Qutip imports
#from qutip import (Qobj, about, basis, coherent, coherent_dm, create, 
#                  destroy,expect, fock, fock_dm, mesolve, qeye, sigmax, 
#                  sigmay,sigmaz, tensor, thermal_dm,dimensions,
#                  rand_dm,rand_unitary,partial_transpose,
#                  hadamard_transform,projection,phase_basis)

import qutip

In [105]:
# Helper function 1
'''
Helper functions - 
all the function that can be used across multiple modules 
and repeatedly often.
''' 

def generate_alpha(delta):
    alpha = random.uniform(0,2*cmath.pi*delta)
    return alpha

def generate_phi(r,s,delta):
    Xi = random.uniform(0,delta)
    phi = math.asin(Xi**(1/(2*r)))
    return phi

def generate_psi(r,s,delta):
    psi = random.uniform(0,2*cmath.pi*delta)
    return psi

def generate_chi(r,s,delta):
    chi = random.uniform(0,2*cmath.pi*delta)
    return chi

def parameter_index(r,s):
    return (s**2-2*r-1, s**2-2*r, (s+1)**2-2*(s+1)) 

In [334]:
# Helper functions 2

def Series_multiply(lst, order = 'normal'):
    Multiplied_list = []
    M = 1
    
    if order=='right':
        lst1 = lst.copy()
        lst1.reverse()
        
        for ls in lst1:
            if isinstance(ls,np.ndarray) and isinstance(M,np.ndarray):
                M = np.matmul(ls,M)
            else:
                M = ls*M
            Multiplied_list.append(M)

        Multiplied_list.reverse()
        
    elif order == 'left':
        for ls in lst:
            if isinstance(ls,np.ndarray) and isinstance(M,np.ndarray):
                M = np.matmul(M,ls)
            else:
                M = M*ls
            Multiplied_list.append(M)
            
    else:
        for ls in lst:
            if isinstance(ls,np.ndarray) and isinstance(M,np.ndarray):
                M = np.matmul(M,ls)
            else:
                M = M*ls
        return M
    
    return Multiplied_list

lst = [(-0.942143429122242-0.33521002216786866j),
 np.array([[-0.3284748 +0.90118739j, -0.16553475-0.22926804j,
         0.        +0.j        ],
       [ 0.16553475-0.22926804j, -0.3284748 -0.90118739j,
         0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         1.        +0.j        ]]),
 np.array([[ 1.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ],
       [ 0.        +0.j        , -0.38865659+0.43426317j,
         0.81262633+0.j        ],
       [ 0.        +0.j        , -0.81262633-0.j        ,
        -0.38865659-0.43426317j]]),
 np.array([[ 0.65932815-0.18281058j,  0.        +0.j        ,
         0.1599924 +0.7115259j ],
       [ 0.        +0.j        ,  1.        +0.j        ,
         0.        +0.j        ],
       [-0.1599924 +0.7115259j ,  0.        +0.j        ,
         0.65932815+0.18281058j]])]

L = Series_multiply(lst,'left')

R = Series_multiply(lst,'right')



In [ ]:
#Rough

0.04683673+0.39598282j
0.04683673+0.39598282j

-0.41910561+0.07066018j
-0.41910561+0.07066018j

0.10086821-0.58856213j
0.10086821-0.58856213j

-0.1486436 -0.07116508j
-0.1486436 -0.07116508j

0.62567052+0.47412786j
0.62567052+0.47412786j

-0.67970304+0.02795727j
-0.67970304+0.02795727j

-0.41939563-0.36957592j,-0.28999122+0.37503091j],
-0.41939563-0.36957592j,-0.28999122+0.37503091j],

In [316]:
#Rough

a = np.array([[1,2],[3,4]])
b = np.array([[8,9],[10,11]])


a = -0.942143429122242-0.33521002216786866j

b = np.array([[-0.3284748 +0.90118739j, -0.16553475-0.22926804j,
         0.        +0.j        ],
       [ 0.16553475-0.22926804j, -0.3284748 -0.90118739j,
         0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         1.        +0.j        ]])

c = np.array([[ 1.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ],
       [ 0.        +0.j        , -0.38865659+0.43426317j,
         0.81262633+0.j        ],
       [ 0.        +0.j        , -0.81262633-0.j        ,
        -0.38865659-0.43426317j]])

lst = [(-0.942143429122242-0.33521002216786866j),
 np.array([[-0.3284748 +0.90118739j, -0.16553475-0.22926804j,
         0.        +0.j        ],
       [ 0.16553475-0.22926804j, -0.3284748 -0.90118739j,
         0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         1.        +0.j        ]]),
 np.array([[ 1.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ],
       [ 0.        +0.j        , -0.38865659+0.43426317j,
         0.81262633+0.j        ],
       [ 0.        +0.j        , -0.81262633-0.j        ,
        -0.38865659-0.43426317j]]),
 np.array([[ 0.65932815-0.18281058j,  0.        +0.j        ,
         0.1599924 +0.7115259j ],
       [ 0.        +0.j        ,  1.        +0.j        ,
         0.        +0.j        ],
       [-0.1599924 +0.7115259j ,  0.        +0.j        ,
         0.65932815+0.18281058j]])]

lst1 = lst.copy()
print(lst1)

[(-0.942143429122242-0.33521002216786866j), array([[-0.3284748 +0.90118739j, -0.16553475-0.22926804j,
         0.        +0.j        ],
       [ 0.16553475-0.22926804j, -0.3284748 -0.90118739j,
         0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         1.        +0.j        ]]), array([[ 1.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ],
       [ 0.        +0.j        , -0.38865659+0.43426317j,
         0.81262633+0.j        ],
       [ 0.        +0.j        , -0.81262633+0.j        ,
        -0.38865659-0.43426317j]]), array([[ 0.65932815-0.18281058j,  0.        +0.j        ,
         0.1599924 +0.7115259j ],
       [ 0.        +0.j        ,  1.        +0.j        ,
         0.        +0.j        ],
       [-0.1599924 +0.7115259j ,  0.        +0.j        ,
         0.65932815+0.18281058j]])]


In [292]:
#Rough

a = np.array([[1,2],[3,4]])
b = np.array([[8,9],[10,11]])
a*(2+9j)==(2+9j)*a

array([[ True,  True],
       [ True,  True]])

In [363]:
# CUE parameter generator function -

def CUE_parameter_generator(size,delta):
    '''
    input: 
    size = N (size of one side of the random unitary matrix 
    being produced), depending on which, this function return a parameter 
    array of size. 
    
    delta = another user defined variable for determining random unitary
    
    return:
    Returns an array of size (n-1)^2 with the structure -
    [alpha,
    
    phi(1,2),
    psi(1,2),
    chi(1,2),
    
    phi(2,3),
    psi(2,3),
    phi(1,3),
    psi(1,3),
    chi(1,3),
    .
    .
    .
    phi(1,n),
    psi(1,n),
    chi(1,n),
    ]
    '''
    
    parameters = []
    parameters.append(('delta',0,1,generate_alpha(delta)))
    for s in range(2,size+1):
        for r in range(s-1,0,-1):
            parameters.append(('phi',r,s,generate_phi(r,s,delta)))
            parameters.append(('psi',r,s,generate_psi(r,s,delta)))
            if r==1:
                parameters.append(('chi',r,s,generate_chi(r,s,delta)))
    return parameters

delta = 1
size = 3
parameters = np.array(CUE_parameter_generator(size,delta))
print(len(parameters))
pprint(parameters)

9
array([['delta', '0', '1', '3.337061352291826'],
       ['phi', '1', '2', '0.25669893061850213'],
       ['psi', '1', '2', '0.8008727539701619'],
       ['chi', '1', '2', '4.209149091097296'],
       ['phi', '2', '3', '1.3636112805092806'],
       ['psi', '2', '3', '4.056347883388843'],
       ['phi', '1', '3', '0.5957225117882995'],
       ['psi', '1', '3', '5.462940924608516'],
       ['chi', '1', '3', '0.5007598961845501']], dtype='<U32')


In [197]:
def Elementary_Unitary_Matrices_Generator(size,r,s,delta,phi, psi, chi):
#     print('elementary',r,s,phi,psi,chi)
    
    # Creating the matrix
    E = np.eye(size, dtype=complex)
    r-=1 
    s-=1
    
    E[r,r] = cmath.cos(phi) * cmath.exp(complex(0,psi))  
    E[r,s] = cmath.sin(phi) * cmath.exp(complex(0,chi))
    E[s,r] = -cmath.sin(phi) * cmath.exp(complex(0,-chi))
    E[s,s] = cmath.cos(phi) * cmath.exp(complex(0,-psi))
    
#     pprint(E)
    return E

n=3
a=1
b=2
delta = 1
phi = generate_phi(a,b,delta)
psi = generate_psi(a,b,delta)
chi = generate_chi(a,b,delta)
    
# Elementary_Unitary_Matrices_Generator(n,a,b,delta,phi,psi,chi)

In [364]:
# CUE_generator - 

# First step is to work with unitary matrices. 
# It will have a function to calculate random unitary matrix using 
# Zyczkowski's method.


def CUE_generator(size, parameters, delta):
    '''
    input: 
    
    1) size = it is the dimensionality (of one side) of the aimed 
    random unitary matrix.
    
    2) parameters = This is a list of parameters used to parametrize the 
    unitary matrix, necessary to generate CUE(Circular Unitary Ensembles)
    deterministically. It is crucial to keep track of these parameters to
    eventually work with gradient descent.
    
    return:
    1) return a unitary matrix of given size, with given parameters and 
    delta.
    '''
    
    print(parameters)
    print(type(parameters[0][3]))
    alpha = parameters[0][3]
    
    Left = []
    Elementary_matrices = [cmath.exp(complex(0,alpha))]
    right = []
    
    for s in range(2,size+1):
        for r in range(s-1,0,-1):
            indexes = parameter_index(r,s)
            phi = float(parameters[indexes[0]][3])
            psi = float(parameters[indexes[1]][3])
            if r==1:
                chi = float(parameters[indexes[2]][3])
            else:
                chi = 0
                
            E = Elementary_Unitary_Matrices_Generator(size,r,s,delta,phi,psi,chi)
            Elementary_matrices.append(E)
            
    Left_multiply = Series_multiply(Elementary_matrices,'left')
    Right_multiply = Series_multiply(Elementary_matrices,'right')
    U = Series_multiply(Elementary_matrices)
    
    return (U,Elementary_matrices, Left_multiply, Right_multiply)
    
delta = 1
size = 3
# parameters = CUE_parameter_generator(size, delta)

CUE_generator(size, parameters, delta)

[['delta' '0' '1' '3.337061352291826']
 ['phi' '1' '2' '0.25669893061850213']
 ['psi' '1' '2' '0.8008727539701619']
 ['chi' '1' '2' '4.209149091097296']
 ['phi' '2' '3' '1.3636112805092806']
 ['psi' '2' '3' '4.056347883388843']
 ['phi' '1' '3' '0.5957225117882995']
 ['psi' '1' '3' '5.462940924608516']
 ['chi' '1' '3' '0.5007598961845501']]
<class 'numpy.str_'>


TypeError: complex() second arg can't be a string

In [132]:
# CUE gradient calculator

def partial_diff(U, parameters, index):
    Elementary = U[1]
    Left = U[2]
    Right = U[3]
    
    symbol = parameters[index][0]
    r = parameters[index][1]
    s = parameters[index][2]
    
    if symbol == 'delta':
        pd = Series_multiply([Left[index-1],Ele])
        
    elif symbol == 'phi':
        phi = parameters[index][3]
        psi = parameters[index+1][3]
        if r==1:
            chi = parameters[index+2][3]
        else:
            chi=0
            
        E = np.eye(size, dtype=complex)
        r-=1 
        s-=1
    
        E[r,r] = -cmath.sin(phi) * cmath.exp(complex(0,psi))  
        E[r,s] = cmath.cos(phi) * cmath.exp(complex(0,chi))
        E[s,r] = -cmath.cos(phi) * cmath.exp(complex(0,-chi))
        E[s,s] = -cmath.sin(phi) * cmath.exp(complex(0,-psi))
        
    elif symbol == 'psi':
        phi = parameters[index-1][3]
        psi = parameters[index][3]
        if r==1:
            chi = parameters[index+1][3]
        else:
            chi=0
            
        E = np.eye(size, dtype=complex)
        r-=1 
        s-=1
    
        E[r,r] = complex(0,1) * cmath.cos(phi) * cmath.exp(complex(0,psi))  
        E[r,s] = cmath.sin(phi) * cmath.exp(complex(0,chi))
        E[s,r] = -cmath.sin(phi) * cmath.exp(complex(0,-chi))
        E[s,s] = cmath.cos(phi) * cmath.exp(complex(0,-psi))
    
    elif symbol == 'chi':
        phi = parameters[index-2][3]
        psi = parameters[index-1][3]
        if r==1:
            chi = parameters[index][3]
        else:
            chi=0
            
        E = np.eye(size, dtype=complex)
        r-=1 
        s-=1
    
        E[r,r] = cmath.cos(phi) * cmath.exp(complex(0,psi))  
        E[r,s] = cmath.sin(phi) * cmath.exp(complex(0,chi))
        E[s,r] = -cmath.sin(phi) * cmath.exp(complex(0,-chi))
        E[s,s] = cmath.cos(phi) * cmath.exp(complex(0,-psi))
    
    return pd

def CUE_gradient(U, parameters):
    Elementary = U[1]
    Left = U[2]
    Right = U[3]
    
    gradient = []
    
    for index in range(len(parameters)):
        gradient.append(partial_diff(U, parameters, index))
        
    return gradient

4

In [338]:
complex(0,1)

1j